# Question 1

In [14]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random


# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)


# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)
dropout = torch.nn.Dropout(p=0.3)


#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)



# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu,dropout,
                            linear3)



# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()




#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)




#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    avg_cost = 0
    model.train()  # set the model to train mode
    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        prediction = model(X)
        loss = criterion(prediction, Y)
        loss.backward()
        optimizer.step()

        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))


print('Learning finished')


#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model.eval()     
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    acc = 0 
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    loss += criterion(prediction, Y_test)
    acc += correct_prediction.float().mean()
    
    print("Accuracy: ", acc.item())


    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Epoch: 0001 cost = 0.496714920
Epoch: 0002 cost = 0.374434590
Epoch: 0003 cost = 0.344285429
Epoch: 0004 cost = 0.312889665
Epoch: 0005 cost = 0.294025153
Epoch: 0006 cost = 0.292008281
Epoch: 0007 cost = 0.279389113
Epoch: 0008 cost = 0.270305425
Epoch: 0009 cost = 0.275044382
Epoch: 0010 cost = 0.280246973
Epoch: 0011 cost = 0.259263664
Epoch: 0012 cost = 0.257747620
Epoch: 0013 cost = 0.249675497
Epoch: 0014 cost = 0.244283572
Epoch: 0015 cost = 0.240869343
Learning finished
Accuracy:  0.9433000087738037
Label:  4
Prediction:  4


# Question 2

In [15]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 128, bias=True)
linear3 = torch.nn.Linear(128, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(512)
bn2 = torch.nn.BatchNorm1d(128)
dropout = torch.nn.Dropout(p=0.3)


#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)



# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)

model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu,dropout,
                            linear3)



# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()




#optimizer 정의하기 (Adam optimizer를 사용할 것!)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)




#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    avg_cost = 0
    model.train()  # set the model to train mode
    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        prediction = model(X)
        loss = criterion(prediction, Y)
        loss.backward()
        optimizer.step()

        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')


#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model.eval()     
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    acc = 0 
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    loss += criterion(prediction, Y_test)
    acc += correct_prediction.float().mean()
    
    print("Accuracy: ", acc.item())


    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Epoch: 0001 cost = 0.411658645
Epoch: 0002 cost = 0.304481298
Epoch: 0003 cost = 0.248990655
Epoch: 0004 cost = 0.233879194
Epoch: 0005 cost = 0.219477236
Epoch: 0006 cost = 0.218943119
Epoch: 0007 cost = 0.199582994
Epoch: 0008 cost = 0.201426744
Epoch: 0009 cost = 0.190604866
Epoch: 0010 cost = 0.178013086
Epoch: 0011 cost = 0.179169849
Epoch: 0012 cost = 0.175442800
Epoch: 0013 cost = 0.166305080
Epoch: 0014 cost = 0.169467181
Epoch: 0015 cost = 0.163018584
Learning finished
Accuracy:  0.871999979019165
Label:  9
Prediction:  9
